In [ ]:
```{r}
# Load necessary libraries and data set file
library(readr)
library(dplyr)
library(ggplot2)
library(tidyr)
library(purrr)
library(broom)
library(infotheo)
library(corrplot)
library(DescTools)
library(epitools)
data <- read.csv("~/Documents/heart_disease_health_indicators_BRFSS2015.csv")
```

```{r}
# Data Cleaning/Pre-processing
str(data)
summary(data)
sum(is.na(data))

# Check the dimension of the dataset
num_records <- nrow(data)
num_columns <- ncol(data)
cat(sprintf("The dataset contains %d records and %d columns.\n", num_records, num_columns))

# Check the attributes in the dataset
cat("Attributes in the dataset:", colnames(data), "\n")

# View the first 5 rows of the dataset
options(width = 120)  # To make sure all columns fit in the display
head(data, 5)
```

```{r}
# Create contingency tables
tab1=xtabs(formula = ~HeartDiseaseorAttack+Stroke,data=data)
print(tab1)

tab2 = xtabs(formula = ~ HeartDiseaseorAttack + Diabetes, data = data)
print(tab2)

tab3 = xtabs(formula = ~ Diabetes + Stroke, data = data)
print(tab3)
```


```{r}
# Subset data for lifestyle factors

# Calculate odds ratios for smoking
odds_ratio_smoker <- oddsratio(x = table(data$HeartDiseaseorAttack, data$Smoker))
print(odds_ratio_smoker)

# Calculate odds ratios for physical activity
odds_ratio_phys_activity <- oddsratio(x = table(data$HeartDiseaseorAttack, data$PhysActivity))
print(odds_ratio_phys_activity)

# Calculate odds ratios for heavy alcohol consumption
odds_ratio_alcohol <- oddsratio(x = table(data$HeartDiseaseorAttack, data$HvyAlcoholConsump))
print(odds_ratio_alcohol)

# Calculate odds ratios for fruit consumption
odds_ratio_fruits <- oddsratio(x = table(data$HeartDiseaseorAttack, data$Fruits))
print(odds_ratio_fruits)
```

```{r}
# Extracting the estimate and CI from the odds ratio calculations correctly
odds_ratios_df <- data.frame(
  Factor = c("Smoking", "Physical Activity", "Heavy Alcohol Consumption", "Fruit Consumption"),
  Odds_Ratio = c(odds_ratio_smoker$measure["1", "estimate"],
                 odds_ratio_phys_activity$measure["1", "estimate"],
                 odds_ratio_alcohol$measure["1", "estimate"],
                 odds_ratio_fruits$measure["1", "estimate"]),
  Lower_95_CI = c(odds_ratio_smoker$measure["1", "lower"],
                  odds_ratio_phys_activity$measure["1", "lower"],
                  odds_ratio_alcohol$measure["1", "lower"],
                  odds_ratio_fruits$measure["1", "lower"]),
  Upper_95_CI = c(odds_ratio_smoker$measure["1", "upper"],
                  odds_ratio_phys_activity$measure["1", "upper"],
                  odds_ratio_alcohol$measure["1", "upper"],
                  odds_ratio_fruits$measure["1", "upper"])
)

# Print the table using knitr::kable()
library(knitr)
kable(odds_ratios_df, format = "markdown", caption = "Odds Ratios and 95% Confidence Intervals for Various Factors Associated with Heart Disease")
```

```{r}
# Create data frame for visualization (life-style factors)
odds_ratios <- data.frame(
  Factor = c("Smoking", "Physical Activity", "Alcohol Consumption", "Fruit Consumption"),
  OddsRatio = c(2.2039, 0.5360, 0.5939, 0.8705),
  LowerCI = c(2.1444, 0.5211, 0.5530, 0.8470),
  UpperCI = c(2.2653, 0.5513, 0.6370, 0.8946)
)

# Plot using ggplot2
library(ggplot2)
ggplot(odds_ratios, aes(x = Factor, y = OddsRatio)) +
  geom_point() +
  geom_errorbar(aes(ymin = LowerCI, ymax = UpperCI), width = 0.2) +
  ggtitle("Odds Ratios and 95% Confidence Intervals for Lifestyle Factors") +
  ylab("Odds Ratio") + xlab("Lifestyle Factor") +
  theme_minimal()
```

```{r}
# Calculate the entropy of the HeartDiseaseorAttack variable
entropy_hd = entropy(data$HeartDiseaseorAttack)
print(paste("Entropy of Heart Disease:", entropy_hd))
```

```{r}
# Conditional entropy of heart disease given smoking
cond_entropy_hd_smoker <- condentropy(data$HeartDiseaseorAttack, data$Smoker)

# Mutual information between heart disease and physical activity
mut_info_hd_activity <- mutinformation(data$HeartDiseaseorAttack, data$PhysActivity)

print(c(entropy_hd, cond_entropy_hd_smoker, mut_info_hd_activity))
```

```{r}
lifestyle_factors <- c("Smoker", "PhysActivity", "HvyAlcoholConsump", "Fruits", "Veggies")

# Initialize a vector to store results
conditional_entropies <- numeric(length(lifestyle_factors))

# Loop through each lifestyle factor to compute conditional entropy
for (i in seq_along(lifestyle_factors)) {
  conditional_entropies[i] <- condentropy(as.factor(data$HeartDiseaseorAttack), as.factor(data[[lifestyle_factors[i]]]))
}

# Combine the results into a data frame for easy viewing
results <- data.frame(Lifestyle_Factor = lifestyle_factors, Conditional_Entropy = conditional_entropies)
print(results)
```

```{r}
# EDA for lifestyle factors
library(ggplot2)

# Prepare the mutual information data for plotting
mi_results <- data.frame(
  Lifestyle_Factor = c("Smoker", "PhysActivity", "HvyAlcoholConsump", "Fruits", "Veggies"),
  Mutual_Information = c(0.006514901, 0.0035234475, 0.0004742529, 0.0001939689, 0.0007280120),
  Colors = c("Smoker", "PhysActivity", "HvyAlcoholConsump", "Fruits", "Veggies")
)

# Define or verify the results data frame
results <- data.frame(
  Lifestyle_Factor = c("Smoker", "PhysActivity", "HvyAlcoholConsump", "Fruits", "Veggies"),
  Conditional_Entropy = c(0.3056014, 0.3085928, 0.3116420, 0.3119223, 0.3113882),  # Example values
  Colors = c("red", "green", "blue", "yellow", "purple")  # Adjust colors as needed
)

# Create the bar plot using Pastel1 colors
library(ggplot2)

ggplot(results, aes(x = Lifestyle_Factor, y = Conditional_Entropy, fill = Lifestyle_Factor)) +
  geom_bar(stat = "identity", width = 0.5) +
  ggtitle("Conditional Entropy of Heart Disease Given Lifestyle Factors") +
  ylab("Conditional Entropy") +
  xlab("Lifestyle Factor") +  # Adding x-label
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),  # Rotate x-axis text labels for better visibility
        legend.title = element_blank(),  # Hides the legend title
        legend.position = "right") +  # Adjust legend position
  scale_fill_brewer(palette = "Pastel1")  # Apply Pastel1 palette
```

```{r}
# Mutual information for lifestyle factors
mutual_information <- numeric(length(lifestyle_factors))

# Loop through each lifestyle factor to compute mutual information
for (i in seq_along(lifestyle_factors)) {
  mutual_information[i] <- mutinformation(as.factor(data$HeartDiseaseorAttack), as.factor(data[[lifestyle_factors[i]]]))
}

# Combine the results into a data frame for easy viewing
mi_results <- data.frame(Lifestyle_Factor = lifestyle_factors, Mutual_Information = mutual_information)
print(mi_results)

#EDA for Mututal information lifestyle factors
library(ggplot2)

# Prepare the data for plotting
mi_results <- data.frame(
  Lifestyle_Factor = c("Smoker", "PhysActivity", "HvyAlcoholConsump", "Fruits", "Veggies"),
  Mutual_Information = c(0.006514901, 0.0035234475, 0.0004742529, 0.0001939689, 0.0007280120)
)

# Create a bar plot for mutual information
ggplot(mi_results, aes(x = Lifestyle_Factor, y = Mutual_Information, fill = Lifestyle_Factor)) +
  geom_bar(stat = "identity", width = 0.5) +
  ggtitle("Mutual Information of Heart Disease With Lifestyle Factors") +
  ylab("Mutual Information") +
  xlab("") +
  theme_minimal() +
  scale_fill_brewer(palette = "Pastel1") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) # Rotate x labels for better visibility
```

```{r}
#Odds ratio for Diabetes with lifestyle factors
# Calculate odds ratios for diabetes and lifestyle factors
odds_ratio_diabetes_smoker <- oddsratio(x = table(data$Diabetes, data$Smoker))
odds_ratio_diabetes_activity <- oddsratio(x = table(data$Diabetes, data$PhysActivity))
odds_ratio_diabetes_alcohol <- oddsratio(x = table(data$Diabetes, data$HvyAlcoholConsump))
odds_ratio_diabetes_fruits <- oddsratio(x = table(data$Diabetes, data$Fruits))

# Data frame for odds ratios related to diabetes
odds_ratios_diabetes_df <- data.frame(
  Factor = c("Smoking", "Physical Activity", "Heavy Alcohol Consumption", "Fruit Consumption"),
  Odds_Ratio = c(odds_ratio_diabetes_smoker$measure["1", "estimate"],
                 odds_ratio_diabetes_activity$measure["1", "estimate"],
                 odds_ratio_diabetes_alcohol$measure["1", "estimate"],
                 odds_ratio_diabetes_fruits$measure["1", "estimate"]),
  Lower_95_CI = c(odds_ratio_diabetes_smoker$measure["1", "lower"],
                  odds_ratio_diabetes_activity$measure["1", "lower"],
                  odds_ratio_diabetes_alcohol$measure["1", "lower"],
                  odds_ratio_diabetes_fruits$measure["1", "lower"]),
  Upper_95_CI = c(odds_ratio_diabetes_smoker$measure["1", "upper"],
                  odds_ratio_diabetes_activity$measure["1", "upper"],
                  odds_ratio_diabetes_alcohol$measure["1", "upper"],
                  odds_ratio_diabetes_fruits$measure["1", "upper"])
)

# Print the table using knitr::kable()
kable(odds_ratios_diabetes_df, format = "markdown", caption = "Odds Ratios and 95% Confidence Intervals for Various Factors Associated with Diabetes")
```

```{r}
# Entropy of Diabetes
entropy_diabetes = entropy(data$Diabetes)
cond_entropy_diabetes_smoker = condentropy(data$Diabetes, data$Smoker)
mut_info_diabetes_activity = mutinformation(data$Diabetes, data$PhysActivity)

# Display entropy and mutual information values
print(paste("Entropy of Diabetes:", entropy_diabetes))
print(c(entropy_diabetes, cond_entropy_diabetes_smoker, mut_info_diabetes_activity))
```

```{r}
# Calculate conditional entropy for all lifestyle factors and diabetes
cond_entropy_diabetes_smoker <- condentropy(data$Diabetes, data$Smoker)
cond_entropy_diabetes_activity <- condentropy(data$Diabetes, data$PhysActivity)
cond_entropy_diabetes_alcohol <- condentropy(data$Diabetes, data$HvyAlcoholConsump)
cond_entropy_diabetes_fruits <- condentropy(data$Diabetes, data$Fruits)
cond_entropy_diabetes_veggies <- condentropy(data$Diabetes, data$Veggies)

# Prepare data frame for visualization of conditional entropy related to diabetes
conditional_entropy_results <- data.frame(
  Lifestyle_Factor = c("Smoker", "PhysActivity", "HvyAlcoholConsump", "Fruits", "Veggies"),
  Conditional_Entropy = c(cond_entropy_diabetes_smoker,
                          cond_entropy_diabetes_activity,
                          cond_entropy_diabetes_alcohol,
                          cond_entropy_diabetes_fruits,
                          cond_entropy_diabetes_veggies)
)
```

```{r}
# EDA for conditional entropy with lifestyle factors/diabetes
library(ggplot2)
ggplot(conditional_entropy_results, aes(x = Lifestyle_Factor, y = Conditional_Entropy, fill = Lifestyle_Factor)) +
  geom_bar(stat = "identity", width = 0.5) +
  ggtitle("Conditional Entropy of Diabetes Given Lifestyle Factors") +
  ylab("Conditional Entropy") +
  xlab("Lifestyle Factor") +
  theme_minimal() +
  scale_fill_brewer(palette = "Pastel1") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))  # Rotate x labels for better visibility
```

```{r}
# Calculate mutual information for all lifestyle factors and diabetes
mut_info_diabetes_smoker <- mutinformation(data$Diabetes, data$Smoker)
mut_info_diabetes_alcohol <- mutinformation(data$Diabetes, data$HvyAlcoholConsump)
mut_info_diabetes_fruits <- mutinformation(data$Diabetes, data$Fruits)
mut_info_diabetes_veggies <- mutinformation(data$Diabetes, data$Veggies)
```

```{r}
# Prepare data frame for visualization of mutual information related to diabetes
mi_results_diabetes <- data.frame(
  Lifestyle_Factor = c("Smoker", "PhysActivity", "HvyAlcoholConsump", "Fruits", "Veggies"),
  Mutual_Information = c(mut_info_diabetes_smoker,
                         mut_info_diabetes_activity,
                         mut_info_diabetes_alcohol,
                         mut_info_diabetes_fruits,
                         mut_info_diabetes_veggies)
)
```

```{r}
# Plot mutual information for diabetes
library(ggplot2)
ggplot(mi_results_diabetes, aes(x = Lifestyle_Factor, y = Mutual_Information, fill = Lifestyle_Factor)) +
  geom_bar(stat = "identity", width = 0.5) +
  ggtitle("Mutual Information of Diabetes With Lifestyle Factors") +
  ylab("Mutual Information") +
  theme_minimal() +
  scale_fill_brewer(palette = "Pastel1") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))  # Rotate x labels for better visibility
```

```{r}
# Lifestyle factors ~ stroke
#Odds ratio
odds_ratio_stroke_smoker <- oddsratio(x = table(data$Stroke, data$Smoker))
odds_ratio_stroke_activity <- oddsratio(x = table(data$Stroke, data$PhysActivity))
odds_ratio_stroke_alcohol <- oddsratio(x = table(data$Stroke, data$HvyAlcoholConsump))
odds_ratio_stroke_fruits <- oddsratio(x = table(data$Stroke, data$Fruits))

odds_ratios_stroke_df <- data.frame(
  Factor = c("Smoking", "Physical Activity", "Heavy Alcohol Consumption", "Fruit Consumption"),
  Odds_Ratio = c(odds_ratio_stroke_smoker$measure["1", "estimate"],
                 odds_ratio_stroke_activity$measure["1", "estimate"],
                 odds_ratio_stroke_alcohol$measure["1", "estimate"],
                 odds_ratio_stroke_fruits$measure["1", "estimate"]),
  Lower_95_CI = c(odds_ratio_stroke_smoker$measure["1", "lower"],
                  odds_ratio_stroke_activity$measure["1", "lower"],
                  odds_ratio_stroke_alcohol$measure["1", "lower"],
                  odds_ratio_stroke_fruits$measure["1", "lower"]),
  Upper_95_CI = c(odds_ratio_stroke_smoker$measure["1", "upper"],
                  odds_ratio_stroke_activity$measure["1", "upper"],
                  odds_ratio_stroke_alcohol$measure["1", "upper"],
                  odds_ratio_stroke_fruits$measure["1", "upper"])
)

# Entropy of Stroke
entropy_stroke = entropy(data$Stroke)

# Conditional entropy for each lifestyle factor
cond_entropy_stroke_smoker = condentropy(data$Stroke, data$Smoker)
cond_entropy_stroke_activity = condentropy(data$Stroke, data$PhysActivity)
cond_entropy_stroke_alcohol = condentropy(data$Stroke, data$HvyAlcoholConsump)
cond_entropy_stroke_fruits = condentropy(data$Stroke, data$Fruits)
cond_entropy_stroke_veggies = condentropy(data$Stroke, data$Veggies)

# Mutual information for each lifestyle factor
mut_info_stroke_smoker = mutinformation(data$Stroke, data$Smoker)
mut_info_stroke_activity = mutinformation(data$Stroke, data$PhysActivity)
mut_info_stroke_alcohol = mutinformation(data$Stroke, data$HvyAlcoholConsump)
mut_info_stroke_fruits = mutinformation(data$Stroke, data$Fruits)
mut_info_stroke_veggies = mutinformation(data$Stroke, data$Veggies)
```

```{r}
# EDA Odds Ratio
# Visualization for Odds Ratios
ggplot(odds_ratios_stroke_df, aes(x = Factor, y = Odds_Ratio, fill = Factor)) +
  geom_bar(stat = "identity", width = 0.5) +  # Display as bars

  ggtitle("Odds Ratios for Stroke by Lifestyle Factor") +  # Title of the graph
  ylab("Odds Ratio") +  # Y-axis label
  xlab("Lifestyle Factor") +  # X-axis label
  theme_minimal() +  # Minimalist theme
  scale_fill_brewer(palette = "Pastel1") +  # Color palette
  theme(axis.text.x = element_text(angle = 45, hjust = 1))  # Rotate x labels for better visibility


# Visualization for Conditional Entropy
conditional_entropy_stroke_results <- data.frame(
  Lifestyle_Factor = c("Smoker", "PhysActivity", "HvyAlcoholConsump", "Fruits", "Veggies"),
  Conditional_Entropy = c(cond_entropy_stroke_smoker,
                          cond_entropy_stroke_activity,
                          cond_entropy_stroke_alcohol,
                          cond_entropy_stroke_fruits,
                          cond_entropy_stroke_veggies)
)

# Print the table using knitr::kable()
knitr::kable(odds_ratios_stroke_df, format = "markdown", caption = "Odds Ratios and 95% Confidence Intervals for Various Factors Associated with Stroke")
```


```{r}
# Subset data for relevant variables for socio-economic factors
data_subset <- data[, c("Stroke", "HeartDiseaseorAttack", "Diabetes", "GenHlth", "MentHlth", "PhysHlth", "DiffWalk", "Sex", "Age", "Education", "Income")]

# Check the structure of the dataset
str(data_subset)

# Check for missing values
sum(is.na(data_subset))

# Drop rows with missing values
data_subset <- na.omit(data_subset)

# Check class imbalance
table(data_subset$HeartDiseaseorAttack)

# Define response variable
Y <- data_subset$Stroke | data_subset$HeartDiseaseorAttack | data_subset$Diabetes

# Fit logistic regression model
model <- glm(Y ~ ., data = data_subset[, -c(1:3)], family = binomial)

# Summarize the model
summary(model)
```

```{r}
# EDA for socio-economic factors
# Plot coefficients
coefficients <- exp(coef(model))
coefficients <- coefficients[-1] # Remove intercept
names(coefficients) <- names(model$coefficients)[-1] # Set names
coefficients <- coefficients[order(coefficients, decreasing = TRUE)] # Order
coef_df <- data.frame(variable = names(coefficients), coefficient = coefficients)
ggplot(coef_df, aes(x = reorder(variable, coefficient), y = coefficient)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  coord_flip() +
  labs(title = "Effect of Socio-Economic Variables on Y",
       x = "Variable",
       y = "Odds Ratio")

# ECDF plot for Age
ggplot(data_subset, aes(x = Age)) +
  stat_ecdf() +
  labs(title = "ECDF of Age",
       x = "Age",
       y = "Cumulative Probability")

# Additional plots for other variables
# ECDF plot for GenHlth
ggplot(data_subset, aes(x = GenHlth)) +
  stat_ecdf() +
  labs(title = "ECDF of General Health",
       x = "General Health",
       y = "Cumulative Probability")

# ECDF plot for MentHlth
ggplot(data_subset, aes(x = MentHlth)) +
  stat_ecdf() +
  labs(title = "ECDF of Mental Health",
       x = "Mental Health",
       y = "Cumulative Probability")

# ECDF plot for PhysHlth
ggplot(data_subset, aes(x = PhysHlth)) +
  stat_ecdf() +
  labs(title = "ECDF of Physical Health",
       x = "Physical Health",
       y = "Cumulative Probability")

# ECDF plot for DiffWalk
ggplot(data_subset, aes(x = DiffWalk)) +
  stat_ecdf() +
  labs(title = "ECDF of Difficulty in Walking",
       x = "Difficulty in Walking",
       y = "Cumulative Probability")

# ECDF plot for Education
ggplot(data_subset, aes(x = Education)) +
  stat_ecdf() +
  labs(title = "ECDF of Education",
       x = "Education Level",
       y = "Cumulative Probability")

# ECDF plot for Income
ggplot(data_subset, aes(x = Income)) +
  stat_ecdf() +
  labs(title = "ECDF of Income",
       x = "Income",
       y = "Cumulative Probability")
```

```{r}
# Odds Ratio and Entropy for socio-economic factors
odds_ratios <- exp(coef(model))
print(odds_ratios)
coefficients <- odds_ratios[-1] # Remove intercept
names(coefficients) <- names(model$coefficients)[-1] # Set names
coefficients <- coefficients[order(coefficients, decreasing = TRUE)] # Order
coef_df <- data.frame(variable = names(coefficients), odds_ratio = coefficients)
ggplot(coef_df, aes(x = reorder(variable, odds_ratio), y = odds_ratio)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  coord_flip() +
  labs(title = "Effect of Socio-Economic Variables on response",
       x = "Variable",
       y = "Odds Ratio")

# Calculating entropy (manually)
entropy <- sapply(data_subset[, -c(1:3)], function(x) {
  p <- table(x) / length(x)
  -sum(p * log2(p))
})
print(entropy)
# Plots for entropy
entropy_df <- data.frame(variable = names(entropy), entropy = entropy)
ggplot(entropy_df, aes(x = reorder(variable, entropy), y = entropy)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  coord_flip() +
  labs(title = "Entropy of Covariates",
       x = "Variable",
       y = "Entropy")
```

```{r}
# Subset of pre-existing conditions
heart
heart2 <- data

# we set values of 2 equal to 1 because they also have diabetes, but a slightly different kind
heart$no_diabetes <- ifelse(heart$Diabetes == 0, 1, 0)
heart$t1_diabetes <- ifelse(heart$Diabetes == 1, 1, 0)
heart$t2_diabetes <- ifelse(heart$Diabetes == 2, 1, 0)

numerical_cols = sapply(heart, is.numeric)
binary_cols = sapply(heart, function(x) is.factor(x) && length(levels(x)) ==2)

identify_binary <- function(column) {
  unique_values <- unique(column)
  all_binary <- all(unique_values %in% c(0, 1))
  return(all_binary)
}

# Identify binary columns
binary_cols <- sapply(heart, identify_binary)

# Extract binary columns
binary_df <- heart[, binary_cols]

# Check the identified binary columns
print(names(binary_df))

# Select numerical columns that are not identified as binary
numerical_cols <- !binary_cols & sapply(heart, is.numeric)

# Separate numerical columns
numerical_df <- heart[, numerical_cols]

# Normalize numerical columns
normalized_numerical_df <- as.data.frame(scale(numerical_df))

# Combine normalized numerical columns with binary columns
normalized_df <- cbind(normalized_numerical_df, binary_df)

# Check the structure of normalized data frame
str(normalized_df)

heart_pre = heart2 %>%
  select(HeartDiseaseorAttack, Stroke, Diabetes, HighBP, HighChol, CholCheck, BMI, Sex, Age, Stroke, GenHlth, MentHlth, PhysHlth)

# descriptive statistics for preexisting variables
summary(heart_pre)
```

```{r}
column_tables <- heart_pre %>%
  gather(column, value) %>%
  group_by(column, value) %>%
  summarise(count = n()) %>%
  ungroup()

tables_list <- split(column_tables, column_tables$column)

# Displaying each table separately
for (table_name in names(tables_list)) {
  cat("Table for column:", table_name, "\n")
  print(tables_list[[table_name]])
  cat("\n")
}

cor_mat = cor(heart_pre)
corrplot(cor_mat, method = "color", type = "upper",
         addCoef.col = "black", number.cex = 0.7,
         tl.col = "black", tl.srt = 45)

### The highest correlated columns are general and physical health. Outside of that, age is pretty high up there with high blood pressure, and high cholesterol.
```



```{r}
# EDA for pre-existing conditions
# Age against heart disease
ggplot(data = heart) +
  geom_histogram(aes(x=Age, fill = factor(HeartDiseaseorAttack))) +
  labs(title = "Age Distribution by Heart Disease or Attack", x = "Age", y = "Count")

# Sex against heart disease
ggplot(data = heart) +
  geom_histogram(aes(x=Sex, fill = factor(HeartDiseaseorAttack))) +
  labs(title = "Sex by Heart Disease or Attack", x = "Sex (0 Female, 1 Male)", y = "Count")

calcs = heart %>%
  group_by(Sex) %>%
  summarize(mean_value = mean(HeartDiseaseorAttack))
print(calcs)


### Men are on average more likely to get heart disease than women (5%)


# Age against heart disease
ggplot(data = heart) +
  geom_histogram(aes(x=Age, fill = factor(HeartDiseaseorAttack))) +
  labs(title = "Age Distribution by Heart Disease or Attack", x = "Age", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=Sex, fill = factor(HeartDiseaseorAttack))) +
  labs(title = "Sex by Heart Disease or Attack", x = "Sex", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=HighBP, fill = factor(HeartDiseaseorAttack))) +
  labs(title = "High Blood Pressure by Heart Disease or Attack", x = "High Blood Pressure", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=HighChol, fill = factor(HeartDiseaseorAttack))) +
  labs(title = "High Cholesterol by Heart Disease or Attack", x = "High Cholesterol", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=Stroke, fill = factor(HeartDiseaseorAttack))) +
  labs(title = "Stroke HIstory by Heart Disease or Attack", x = "Stroke HIstory", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=GenHlth, fill = factor(HeartDiseaseorAttack))) +
  labs(title = "General Health by Heart Disease or Attack", x = "General Health", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=Age, fill = factor(Diabetes))) +
  labs(title = "Age Distribution by Diabetes", x = "Age", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=HighBP, fill = factor(Diabetes))) +
  labs(title = "High Blood Pressure by Diabetes", x = "High Blood Pressure", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=HighChol, fill = factor(Diabetes))) +
  labs(title = "High Cholesterol by Diabetes", x = "High Cholesterol", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=Stroke, fill = factor(Diabetes))) +
  labs(title = "Stroke HIstory by Diabetes", x = "Stroke HIstory", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=GenHlth, fill = factor(Diabetes))) +
  labs(title = "General Health by Diabetes", x = "General Health", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=BMI, fill = factor(Diabetes))) +
  labs(title = "BMI by Diabetes", x = "BMI", y = "Count")

tab1 = xtabs(formula = ~Diabetes+HighBP, data = heart)
print(addmargins(tab1))
print("------------")
tab1 = xtabs(formula = ~Diabetes+HighChol, data = heart)
print(addmargins(tab1))

ggplot(data = heart) +
  geom_histogram(aes(x=Age, fill = factor(Stroke))) +
  labs(title = "Age Distribution by Stroke", x = "Age", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=Sex, fill = factor(Stroke))) +
  labs(title = "Sex by Stroke", x = "Sex", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=HighBP, fill = factor(Stroke))) +
  labs(title = "High Blood Pressure by Stroke", x = "High Blood Pressure", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=HighChol, fill = factor(Stroke))) +
  labs(title = "High Cholesterol by Stroke", x = "High Cholesterol", y = "Count")

ggplot(data = heart) +
  geom_histogram(aes(x=BMI, fill = factor(Stroke))) +
  labs(title = "BMI by Stroke", x = "BMI", y = "Count")


tab1 = xtabs(formula = ~Stroke+HighBP, data = heart)
print(addmargins(tab1))
print("------------")
tab1 = xtabs(formula = ~Stroke+HighChol, data = heart)
print(addmargins(tab1))

# 1.8% of people without high blood pressure have a stroke, 7.5% with high blood pressure have a stroke
# 2.6% of people without high cholesterol have a stroke, 6.6% of people with high cholesterol have a stroke


count <- heart_pre %>%
  filter(HighBP == 1, HighChol == 1) %>%
  nrow()

print(paste0("Number of people with both high blood pressure and cholesterol:", count))
print(paste0("Number of total records in the data:", nrow(heart_pre)))
print(sum(heart_pre$HighBP))
print(sum(heart_pre$HighChol))


### of the approximately 108K people who have high blood pressure or high cholesterol, 60K of them in fact have both, and overlap in a venn diagram sense. Each category here has approximately 40K people that do not have the other condition

model = glm(HeartDiseaseorAttack ~ HighBP + HighChol + CholCheck + Sex + Age, data = heart, family = "binomial")

summary(model)


### blood pressure is pretty big, cholesterol less so, sex is also weirdly kind of indicative with some good numbers to back it up


model = glm(t1_diabetes ~ HighBP + HighChol + CholCheck + Sex + Age, data = heart, family = "binomial")

summary(model)

model = glm(t2_diabetes ~ HighBP + HighChol + CholCheck + Sex + Age, data = heart, family = "binomial")

summary(model)


model = glm(Stroke ~ HighBP + HighChol + CholCheck + Sex + Age, data = heart, family = "binomial")

summary(model)

sum(is.na(heart_pre))

table(heart_pre$HeartDiseaseorAttack)


Y = heart_pre$HeartDiseaseorAttack | heart_pre$Stroke | heart_pre$Diabetes

model = glm(Y ~ ., data = heart_pre[, -c(1:3)])

summary(model)

model = glm(Stroke ~ HighBP + HighChol + CholCheck + Sex + Age, data = heart, family = "binomial")

summary(model)


sum(is.na(heart_pre))

table(heart_pre$HeartDiseaseorAttack)


Y = heart_pre$HeartDiseaseorAttack | heart_pre$Stroke | heart_pre$Diabetes

model = glm(Y ~ ., data = heart_pre[, -c(1:3)])

summary(model)
```
